# Подготовка датасета для соревнования [SF-DST] Car Price prediction
В данном ноутбуке мы преобразуем ранее выгруженные с авто.ру json-файлы в датасет, который подгрузим на kaggle

In [1]:
import json
import pandas as pd

Откроем 2 ранее выгруженных json-файла и сделаем из них один.

In [2]:
with open("data.json", "r", encoding="utf8") as f:
    data = json.load(f)
df = pd.DataFrame(data)

with open("data2.json", "r", encoding="utf8") as f:
    data = json.load(f)
df2 = pd.DataFrame(data)

df = df.append(df2, sort=True, ignore_index=True)

Посмотрим на получившийся датасет. Также нам поможет JSON Viewer: http://jsonviewer.stack.hu/

In [3]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6753 entries, 0 to 6752
Data columns (total 39 columns):
additional_info      6753 non-null object
autostrategies       6753 non-null object
availability         6753 non-null object
badges               41 non-null object
brand_cert_info      6753 non-null object
category             6753 non-null object
color_hex            6753 non-null object
counters             6753 non-null object
credit_info          6753 non-null object
daily_counters       6753 non-null object
delivery_info        6753 non-null object
description          6563 non-null object
discount_options     147 non-null object
discount_price       35 non-null object
documents            6753 non-null object
grouping_info        0 non-null object
hash                 6753 non-null object
id                   6753 non-null object
is_favorite          6753 non-null bool
lk_summary           6753 non-null object
old_category_id      6753 non-null int64
owner_expenses       6

,additional_info,autostrategies,availability,badges,brand_cert_info,category,color_hex,counters,credit_info,daily_counters,...,seller_type,service_prices,service_schedules,services,soldMinutes,state,status,sub_category,tags,vehicle_info
0,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,NaN,{},cars,040001,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},"[{'service': 'all_sale_special', 'is_active': ...",NaN,"{'mileage': 195000, 'state_not_beaten': True, ...",ACTIVE,cars,"[good_price, oversize, medium, fivestars, pres...","{'configuration': {'id': '4601199', 'body_type..."
1,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,NaN,{},cars,040001,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},"[{'service': 'all_sale_special', 'is_active': ...",NaN,"{'mileage': 219000, 'state_not_beaten': True, ...",ACTIVE,cars,"[good_price, big, medium, fivestars, prestige,...","{'configuration': {'id': '6088623', 'body_type..."
2,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,"[Обслуживалась у дилера, Оригинальный пробег, ...",{},cars,97948F,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},"[{'service': 'all_sale_special', 'is_active': ...",NaN,"{'mileage': 155000, 'state_not_beaten': True, ...",ACTIVE,cars,"[good_price, big, oversize, discount, medium, ...","{'configuration': {'id': '2305680', 'body_type..."
3,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,NaN,{},cars,200204,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},[],NaN,"{'mileage': 348000, 'state_not_beaten': True, ...",ACTIVE,cars,"[oversize, medium, fivestars, prestige, comfor...","{'configuration': {'id': '4601199', 'body_type..."
4,"{'hidden': False, 'is_on_moderation': False, '...",[],IN_STOCK,"[Один владелец, Максимальная комплектация, В и...",{},cars,040001,{},{'state': 'NOT_AVALIABLE'},[],...,PRIVATE,[],{},"[{'service': 'all_sale_special', 'is_active': ...",NaN,"{'mileage': 150000, 'state_not_beaten': True, ...",ACTIVE,cars,"[proven_owner, big, prestige, comfort, badge, ...","{'configuration': {'id': '3619826', 'body_type..."


Посмотрим, какие данные мы сможем забрать. Начнем по порядку.

## Additional Info

In [4]:
df.additional_info[0]

{'hidden': False,
 'is_on_moderation': False,
 'not_disturb': True,
 'update_date': '1594724311000',
 'creation_date': '1594289299898',
 'mobile_autoservices_url': 'https://m.auto.ru/autoservice/all_works/BMW/?geo_id=213',
 'hot_info': {'is_hot': False,
  'start_time': '2020-07-09T10:05:52Z',
  'end_time': '2020-07-09T10:05:52Z'},
 'fresh_date': '1594724292980',
 'counters_start_date': '1594289299898',
 'chat_only': False,
 'dealer_card_promo': 'DONT_SHOW',
 'review_summary': {'avg_rating': 4.599999904632568, 'counter': 62},
 'price_stats': {'last_year_price_percentage_diff': -14},
 'booking': {}}

Здесь может пригодиться рейтинг авто. Сделаем такой признак.

## Rating

In [5]:
df['rating'] = df['additional_info'].apply(lambda x: x.get('review_summary').get('avg_rating'))

Обработаем пропуски

In [6]:
df.rating.isnull().sum()

262

In [7]:
print(df.rating.value_counts())
df.rating.mean()

4.6    2533
4.8    1450
4.7     957
4.5     925
4.4     170
4.3     129
4.9     114
5.0      95
4.2      68
3.9      35
4.1       6
3.3       6
3.8       2
3.7       1
Name: rating, dtype: int64


4.63507932423627

Заменим пропуски на 4.6 т.к. это значение самое частое и близко к среднему.

In [8]:
df.rating = df.rating.fillna(4.6)

## priceDiff

Возьмем ещё информацию по изменению цены по сравнению с прошлым годом.

In [9]:
df['priceDiff'] = df['additional_info'].apply(lambda x: x.get('price_stats').get('last_year_price_percentage_diff'))

Обработаем пропуски

In [10]:
df['priceDiff'].isnull().sum()

130

In [11]:
print(df.priceDiff.value_counts())
df.priceDiff.mean()

-8.0     952
-11.0    867
-14.0    843
-13.0    756
-9.0     590
-6.0     566
-10.0    510
-15.0    481
-17.0    316
-12.0    251
-1.0     127
-4.0     109
-16.0     86
-19.0     52
-3.0      46
-20.0     36
-7.0      35
Name: priceDiff, dtype: int64


-10.904121999094066

Заменим пропуски на -11 т.к. близко к среднему и часто встречается.

In [12]:
df.priceDiff = df.priceDiff.fillna(-11.0)

Удалим 'additional_info'

In [13]:
df = df.drop('additional_info', axis=1)

## Autostrategies
Тут пусто, удалим

In [14]:
df = df.drop('autostrategies', axis=1)

## Availability

In [15]:
df.availability.value_counts()

IN_STOCK    6751
ON_ORDER       2
Name: availability, dtype: int64

Видимо, некоторые салоны привозят авто под заказ. Не думаю, что это сильно повлияет на цену. Удалим этот признак.

In [16]:
df = df.drop('availability', axis=1)

## Badges, brand_cert_info, category
Ничего интересного, удалаяем

In [17]:
for columns in [
    'badges',
    'brand_cert_info',
    'category',
]:
    df = df.drop(columns, axis=1)

## Color_hex
Цвет машины, записан в виде кодировки RGB. Переделаем в название цветов на английском.

In [18]:
df.color_hex.value_counts()

040001    2733
FAFBFB    1134
0000CC     845
97948F     788
CACECB     424
200204     257
EE1D19     200
007F00     106
22A0F8      72
C49648      61
660099      40
DEA522      38
FF8649      29
4A2197      20
FFD600       5
FFC0CB       1
Name: color_hex, dtype: int64

In [19]:
dic_color = {
    '040001': 'black',
    'FAFBFB': 'white',
    '97948F': 'grey',
    '0000CC': 'dark_blue',
    'CACECB': 'silver', 
    'EE1D19': 'red',
    '200204': 'brown',
    '007F00': 'green',
    'C49648': 'biege',
    '22A0F8': 'blue',
    '660099': 'purple',
    'FF8649': 'orange',
    'DEA522': 'golden',
    '4A2197': 'violet',
    'FFD600': 'yellow', 
}
df['color_hex'] = df['color_hex'].map(dic_color)
df.rename(columns = {'color_hex':'color'}, inplace = True) 

Посмотрим на пропуски.

In [20]:
df.color.isnull().sum()

1

Пусть будет черным.

In [21]:
df.color = df.color.fillna('black')

## Counters, credit_info, daily_counters, delivery_info 
Удаляем

In [22]:
for columns in [
    'counters',
    'credit_info',
    'daily_counters',
    'delivery_info',
]:
    df = df.drop(columns, axis=1)

## Description

In [23]:
df.description[0]

'Автомобиль в отличном состоянии. Ни каких нареканий по лакокрасочному покрытию. Салон чистый и ухоженный, не прокурен. Двигатель работает в штатном режиме, не дымит и не троит. Коробка переключает плавно и без рывков. Отсутствуют посторонние шумы в ходовой части. Машина прошла предпродажную диагностику ДВС и АКПП, в результате которой, не выявлено никаких  отклонений. \n-А также вы можете приобрести данный автомобиль в кредит!!!\n-Рассмотрим вариант обмена!!!\n-Гарантируем юридическую чистоту автомобиля!!!'

Интересно было бы проанализировать, но вряд ли успеем в срок. Удаляем.

In [24]:
df = df.drop('description', axis=1)

## Discount_options, discount_price
Не очень интересно и много пропусков. Удаляем.

In [25]:
df = df.drop('discount_options', axis=1)
df = df.drop('discount_price', axis=1)

## Documents

In [26]:
df.documents[0]

{'owners_number': 3,
 'pts_original': True,
 'custom_cleared': True,
 'year': 2004,
 'vin': 'WBA**************',
 'pts': 'ORIGINAL',
 'license_plate': '******|152',
 'vin_resolution': 'OK'}

Отсюда мы можем взять количество владельцев, ПТС оригинал или дубликат и год выпуска.

### Количество владельцев - owners

Забираем данные о количестве владельцев:

In [27]:
df['owners'] = df['documents'].apply(lambda x: x.get('owners_number'))

In [28]:
df.owners.value_counts()

3.0    3313
2.0    1758
1.0    1532
Name: owners, dtype: int64

In [29]:
print('Количество пропусков: {},'.format(df.owners.isnull().sum()))

Количество пропусков: 150,


Видим, что довольно много пропусков. Скорее всего тут можно заменить пропуски, используя возраст машины. Но у нас пока нет этого параметра.

### Данные о ПТС - technicalPassport

Забираем данные о ПТС:

In [30]:
df['technicalPassport'] = df['documents'].apply(lambda x: x.get('pts'))

In [31]:
df.technicalPassport.value_counts()

ORIGINAL     5741
DUPLICATE    1010
Name: technicalPassport, dtype: int64

In [32]:
print('Количество пропусков: {},'.format(df.technicalPassport.isnull().sum()))

Количество пропусков: 2,


Отлично, всего 2 пропуска заменим их на original. Приведем значения к виду как на kaggle.

In [33]:
dic_technicalPassport = {
    'ORIGINAL': 1,
    'DUPLICATE': 0
}
df['technicalPassport'] = df['technicalPassport'].map(dic_technicalPassport)

In [34]:
df.technicalPassport = df.technicalPassport.fillna(1)

### Возраст машины - age

Забираем данные о годе выпуска машины по документам:

In [35]:
df['age'] = df['documents'].apply(lambda x: x.get('year'))

In [36]:
df.age.value_counts()

2013    599
2011    566
2012    561
2008    486
2019    444
2010    381
2018    366
2014    345
2017    341
2009    325
2016    309
2007    308
2015    262
2004    175
2005    158
2006    149
2003    136
2001    126
2002    122
1998     90
1999     81
1997     79
2000     75
1996     37
1991     31
1993     31
1990     27
1992     25
1994     23
1995     20
1988     14
1989     12
1987      8
1984      6
1985      5
1986      4
1937      4
1980      3
1983      3
1982      3
1950      2
1972      1
1968      1
1953      1
1975      1
1949      1
1978      1
1969      1
1981      1
1947      1
1938      1
1948      1
Name: age, dtype: int64

In [37]:
df['age']= 2020-df['age']

Удалим автомобили старше 38 лет (как в тестовой выборке)

In [38]:
df = df[df.age < 39]

Есть ли пропуски?

In [39]:
print('Количество пропусков: {},'.format(df.age.isnull().sum()))

Количество пропусков: 0,


Здесь мы все что нужно забрали. Удалим столбец.

In [40]:
df = df.drop('documents', axis=1)

## Вернемся к пропускам в owners

In [41]:
print('Количество пропусков: {},'.format(df.owners.isnull().sum()))

Количество пропусков: 150,


Посмотрим на возраст автомобиля в случаях, когда нет информации о количестве владельцев.

In [42]:
df[df.owners.isnull()==True].age.value_counts()

1     139
2       7
15      2
17      1
3       1
Name: age, dtype: int64

Видим, что почти все автомобили относительно новые. Заменим количество владельцев в пропусках на 1.

In [43]:
df.owners = df.owners.fillna(1)

## grouping_info, hash, id, is_favorite, lk_summary, lk_summary, old_category_id, price_history

Удаляем т.к.:
* grouping_info - пусто
* hash - не интересно
* id - не интересно
* is_favorite - не интересно
* lk_summary - содержит информацию, которая есть в других столбцах
* old_category_id - не интересно
* price_history - нас интересует только последняя цена

In [44]:
for columns in [
    'grouping_info',
    'hash',
    'id',
    'is_favorite',
    'lk_summary',
    'old_category_id',
    'price_history',
]:
    df = df.drop(columns, axis=1)

## owner_expences

In [45]:
df.owner_expenses[25]

{'transport_tax': {'tax_by_year': 42900,
  'year': 2020,
  'rid': 213,
  'rate': 150,
  'horse_power': 286,
  'holding_period_month': 12,
  'boost': 1},
 'osago_insurance': [{'insurance_company': 'TINKOFF', 'price': 6309}]}

Отсюда мы можем взять величину транспортного налога и стоимость ОСАГО.

## tax_price

In [46]:
df['taxPrice'] = df['owner_expenses'].apply(lambda x: x.get('transport_tax'))

Посмотрим на количество пропусков.

In [47]:
df['taxPrice'].isnull().sum()

1

Удалим пропуск и запишем в колонку сумму налога.

In [48]:
df = df[df['taxPrice'].isnull()==False]

In [49]:
df['taxPrice'] = df['taxPrice'].apply(lambda x: x.get('tax_by_year'))

## insurance_price

In [50]:
df['tax_insurance_price'] = df['owner_expenses'].apply(lambda x: x.get('osago_insurance'))

Посмотрим на количество пропусков:

In [51]:
df['tax_insurance_price'].isnull().sum()

16

Не очень много. Удалим пропуски и запишем в колонку стоимость ОСАГО

In [52]:
df = df[df['tax_insurance_price'].isnull()==False]

In [53]:
df['insurancePrice'] = df['tax_insurance_price'].apply(lambda x: x[0].get('price'))

Удалим 'owner_expenses' и 'tax_insurance_price'

In [54]:
df = df.drop('owner_expenses', axis=1)
df = df.drop('tax_insurance_price', axis=1)

## Price_info
Цена в разных валютах

Добавим информацию в наш датафрейм.

In [55]:
df['price'] = df['price_info'].apply(lambda x: x.get('RUR'))
df['price_usd'] = df['price_info'].apply(lambda x: x.get('USD'))
df['price_eur'] = df['price_info'].apply(lambda x: x.get('EUR'))

In [56]:
df['price'].mean()/df['price_usd'].mean()

71.12754291219072

Похоже на сегодняшний курс. Логично, что авто.ру сам переводит рублевые цены в доллары.
Удалим цены в зарубежных валютах и price_info.

In [57]:
for columns in [
    'price_info',
    'price_usd',
    'price_eur',
]:
    df = df.drop(columns, axis=1)

Пропуски?

In [58]:
print('Количество пропусков: {},'.format(df.price.isnull().sum()))

Количество пропусков: 0,


т.к. это целевая переменная, заполнять пропуске будет не правильно. Удалим эти значения, их не много.

In [59]:
df = df[df.price.isnull()==False]

### recall_info, saleId, salon, section, seller, seller_type, service_prices, service_schedules, services, soldMinutes, status, sub_category
Либо пусто, либо ничего полезного

In [60]:
for columns in [
    'recall_info',
    'saleId',
    'salon',
    'section',
    'seller',
    'seller_type',
    'service_prices',
    'service_schedules',
    'services',
    'soldMinutes',
    'status',
    'sub_category',
]:
    df = df.drop(columns, axis=1)

## State
Состояние. Отсюда мы можем взять пробег.

In [61]:
df['mileage'] = df['state'].apply(lambda x: x.get('mileage'))

Есть ли пропуски?

In [62]:
print('Количество пропусков: {},'.format(df.mileage.isnull().sum()))

Количество пропусков: 0,


Здесь ещё есть информация о том, является ли автомобиль новым или с пробегом. Но в тестовой выборке все авто с пробегом и сделали такой фильтр на этапе парсинга.

In [63]:
df = df.drop('state', axis=1)

## tags
Тэги. Думаю, что тэги будут неплохими дополнительными параметрами для нашей модели.

In [64]:
df.tags[0]

['good_price',
 'oversize',
 'medium',
 'fivestars',
 'prestige',
 'comfort',
 'allowed_for_credit',
 'autoru_exclusive',
 'wide-back-seats',
 'big-trunk',
 'all-terrain',
 'vin_offers_history',
 'options',
 'available_for_checkup',
 'handling',
 'vin_resolution_ok',
 'vin_checked',
 'pts_original',
 'equitable_price',
 'high_reviews_mark']

Вот только непонятно, как мы обогатим этими парметрами данные из тестовой выборки. Удалим столбец, но в будущем подумаем о возможных тэгах.

In [65]:
df = df.drop('tags', axis=1)

## vehicle_info
Информация об автомобиле

In [66]:
df.vehicle_info[25]

{'configuration': {'id': '2305608',
  'body_type': 'SEDAN',
  'doors_count': 4,
  'auto_class': 'E',
  'human_name': 'Седан',
  'trunk_volume_min': 520,
  'body_type_group': 'SEDAN',
  'main_photo': {'sizes': {'orig': '//avatars.mds.yandex.net/get-verba/787013/2a000001609cfc38f5dc8d47523a92d90eca/orig',
    'wizardv3mr': '//avatars.mds.yandex.net/get-verba/787013/2a000001609cfc38f5dc8d47523a92d90eca/wizardv3mr',
    'cattouch': '//avatars.mds.yandex.net/get-verba/787013/2a000001609cfc38f5dc8d47523a92d90eca/cattouch'}},
  'tags': ['big', 'big-trunk', 'wide-back-seats', 'medium']},
 'complectation': {'id': '4720222',
  'name': '535',
  'available_options': ['cruise-control',
   'airbag-passenger',
   'lock',
   'electro-mirrors',
   'mirrors-heat',
   'leather',
   'computer',
   'wheel-power',
   'airbag-side',
   'abs',
   'climate-control-1',
   'esp',
   'audiopreparation',
   'electro-window-back',
   'condition',
   'airbag-driver',
   'electro-window-front',
   'ptf',
   'audiosys

Пойдем по порядку

### body_Type
Тип кузова. Приведем в вид, аналогичный тестовому датасету на Kaggle.

In [67]:
df['bodyType'] = df['vehicle_info'].apply(lambda x: x.get('configuration').get('body_type'))

In [68]:
df.bodyType.value_counts()

SEDAN                3037
ALLROAD_5_DOORS      2696
COUPE                 340
HATCHBACK_5_DOORS     239
LIFTBACK              159
WAGON_5_DOORS          82
HATCHBACK_3_DOORS      72
CABRIO                 57
ROADSTER               26
COMPACTVAN              5
COUPE_HARDTOP           2
SEDAN_2_DOORS           1
Name: bodyType, dtype: int64

In [69]:
dic_bodyType = {
    'SEDAN': 'sedan',
    'SEDAN_2_DOORS': 'sedan',
    'ALLROAD_5_DOORS': 'SUV',
    'HATCHBACK_5_DOORS': 'hatchback',
    'HATCHBACK_3_DOORS': 'hatchback',
    'COUPE': 'coupe',
    'COUPE_HARDTOP': 'coupe',    
    'ROADSTER': 'roadster',
    'LIFTBACK': 'liftback',
    'WAGON_5_DOORS': 'wagon',
    'CABRIO': 'cabriolet',
    'COMPACTVAN': 'van'
}
df['bodyType'] = df['bodyType'].map(dic_bodyType)

Пропуски?

In [70]:
df['bodyType'].isnull().sum()

0

### doors_count
Количество дверей

In [71]:
df['numberOfDoors'] = df['vehicle_info'].apply(lambda x: x.get('configuration').get('doors_count'))

In [72]:
df.numberOfDoors.value_counts()

5    3181
4    3037
2     426
3      72
Name: numberOfDoors, dtype: int64

Есть ли пропуски?

In [73]:
df['numberOfDoors'].isnull().sum()

0

В ноутбуке на Kaggle решили, что количество дверей скорее определяет категорию автомобиля. Преобразуем значения в строку.

In [74]:
df['numberOfDoors'] = df['numberOfDoors'].apply(lambda s: str(int(s)))

### Auto_class
Класс автомобиля. Добавим этот признак.

In [75]:
df['autoClass'] = df['vehicle_info'].apply(lambda x: x.get('configuration').get('auto_class'))

Пропуски?

In [76]:
df['autoClass'].isnull().sum()

0

### Complectation
Комплектация

In [77]:
df.vehicle_info[25].get('complectation')

{'id': '4720222',
 'name': '535',
 'available_options': ['cruise-control',
  'airbag-passenger',
  'lock',
  'electro-mirrors',
  'mirrors-heat',
  'leather',
  'computer',
  'wheel-power',
  'airbag-side',
  'abs',
  'climate-control-1',
  'esp',
  'audiopreparation',
  'electro-window-back',
  'condition',
  'airbag-driver',
  'electro-window-front',
  'ptf',
  'audiosystem-cd',
  'wheel-configuration2',
  'wheel-configuration1',
  'immo']}

Заберем список опций (available_options) в виде списка. Далее сделаем dummy-переменные.

In [78]:
df['complectation'] = df['vehicle_info'].apply(lambda x: x.get('complectation').get('available_options'))

In [79]:
df['complectation'][25]

['cruise-control',
 'airbag-passenger',
 'lock',
 'electro-mirrors',
 'mirrors-heat',
 'leather',
 'computer',
 'wheel-power',
 'airbag-side',
 'abs',
 'climate-control-1',
 'esp',
 'audiopreparation',
 'electro-window-back',
 'condition',
 'airbag-driver',
 'electro-window-front',
 'ptf',
 'audiosystem-cd',
 'wheel-configuration2',
 'wheel-configuration1',
 'immo']

Проверим количество пропусков

In [80]:
print('Количество пропусков: {},'.format(df['complectation'].isnull().sum()))

Количество пропусков: 4593,


Очень много пропусков. Удалим этот признак т.к. в признаке 'equipment' также будут перечислены различные опции.

In [81]:
df = df.drop('complectation', axis=1)

### Equipment
Оборудование

In [82]:
df.vehicle_info[25].get('equipment')

{'cruise-control': True,
 'asr': True,
 'tinted-glass': True,
 'esp': True,
 'adaptive-light': True,
 'usb': True,
 'sport-seats': True,
 'e-adjustment-wheel': True,
 'multi-wheel': True,
 'xenon': True,
 'audiosystem-tv': True,
 'ashtray-and-cigarette-lighter': True,
 'airbag-passenger': True,
 'front-centre-armrest': True,
 'navigation': True,
 'front-seat-support': True,
 'decorative-interior-lighting': True,
 'isofix-front': True,
 'lock': True,
 'door-sill-panel': True,
 'electro-mirrors': True,
 'drl': True,
 'electro-window-back': True,
 'reduce-spare-wheel': True,
 'multizone-climate-control': True,
 'mirrors-heat': True,
 'power-latching-doors': True,
 'hatch': True,
 'volume-sensor': True,
 'dark-interior': True,
 'sport-suspension': True,
 'folding-front-passenger-seat': True,
 'leather': True,
 '19-inch-wheels': True,
 'driver-seat-memory': True,
 'sport-pedals': True,
 'engine-proof': True,
 'front-seats-heat-vent': True,
 'start-stop-function': True,
 'airbag-driver': Tru

Преобразуем словарь в список ключей и запишем в переменную equioment.

In [83]:
df['equipment'] = df['vehicle_info'].apply(lambda x: list(x.get('equipment').keys()))

In [84]:
df.equipment[25]

['cruise-control',
 'asr',
 'tinted-glass',
 'esp',
 'adaptive-light',
 'usb',
 'sport-seats',
 'e-adjustment-wheel',
 'multi-wheel',
 'xenon',
 'audiosystem-tv',
 'ashtray-and-cigarette-lighter',
 'airbag-passenger',
 'front-centre-armrest',
 'navigation',
 'front-seat-support',
 'decorative-interior-lighting',
 'isofix-front',
 'lock',
 'door-sill-panel',
 'electro-mirrors',
 'drl',
 'electro-window-back',
 'reduce-spare-wheel',
 'multizone-climate-control',
 'mirrors-heat',
 'power-latching-doors',
 'hatch',
 'volume-sensor',
 'dark-interior',
 'sport-suspension',
 'folding-front-passenger-seat',
 'leather',
 '19-inch-wheels',
 'driver-seat-memory',
 'sport-pedals',
 'engine-proof',
 'front-seats-heat-vent',
 'start-stop-function',
 'airbag-driver',
 'aux',
 'isofix',
 'electro-window-front',
 'projection-display',
 'light-sensor',
 'automatic-lighting-control',
 'passenger-seat-updown',
 'computer',
 'high-beam-assist',
 'keyless-entry',
 'seat-transformation',
 'wheel-power',
 'al

Пропуски?

In [85]:
df['equipment'].isnull().sum()

0

### Tags
Эти тэги нам уже больше подойдут т.к. относятся к модели машины.

In [86]:
df['tags'] = df['vehicle_info'].apply(lambda x: x.get('configuration').get('tags'))

Есть ли пропуски?

In [87]:
df['tags'].isnull().sum()

4

Пропусков мало. Удалим эти значения.

In [88]:
df = df[df['tags'].isnull()==False]

В дальнейшем можем сделать из тэгов dummy-переменные

### model_info
Название модели

In [89]:
df['model'] = df['vehicle_info'].apply(lambda x: x.get('model_info').get('code'))

In [90]:
df.model.value_counts()

5ER              1651
3ER              1142
X5               1048
7ER               550
X6                520
X3                445
X1                318
1ER               308
6ER               141
X4                106
X6_M               97
X7                 80
4                  78
X5_M               60
M5                 39
Z4                 21
M3                 19
X2                 18
M6                 15
2ER                10
8ER                 8
I3                  6
I8                  5
M4                  5
2GRANDTOURER        4
Z3                  4
M2                  4
X4_M                2
M8                  2
X3_M                2
Z4_M                1
2ACTIVETOURER       1
Z8                  1
Z1                  1
Name: model, dtype: int64

Пропуски?

In [91]:
df['model'].isnull().sum()

0

Этот параметр сильно влияет на стоимость автомобиля и его нет в тестовой выборке. После формировния датасета попробуем найти зависимости, чтобы можно было заполнить эти значения в тестовом датасете. Займемся этим после получения остальных признаков.

### super_gen
Поколение. Думаю, что будет сильно коррелировать с годом производства. Скорее всего от поколения цена сильно зависит. Заберем отсюда год начала выпуска модели как в тестовой выборке.

In [92]:
df['vehicle_info'][0].get('super_gen')

{'id': '4601198',
 'name': 'I (E53) Рестайлинг',
 'year_from': 2003,
 'year_to': 2006,
 'price_segment': 'PREMIUM'}

In [93]:
df['modelAge'] = 2020 - df['vehicle_info'].apply(lambda x: x.get('super_gen').get('year_from'))

Пропуски?

In [94]:
df['modelAge'].isnull().sum()

0

### tech_param
Здесь много полезной информации

In [95]:
df.vehicle_info[25].get('tech_param')

{'id': '2305626',
 'name': '535',
 'nameplate': '535d',
 'displacement': 2993,
 'engine_type': 'DIESEL',
 'gear_type': 'REAR_DRIVE',
 'transmission': 'AUTOMATIC',
 'power': 286,
 'power_kvt': 210,
 'human_name': '535d 3.0d AT (286 л.с.)',
 'acceleration': 6.4,
 'clearance_min': 142,
 'fuel_rate': 6.7}

#### name

In [96]:
df['name'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('name'))

Посмотрим на значения

In [97]:
df['name'].value_counts()

520     659
320     530
30d     466
35i     314
20d     280
       ... 
35is      1
524       1
535L      1
725       1
728L      1
Name: name, Length: 105, dtype: int64

Посмотрим на пропуски.

In [98]:
df['name'].isnull().sum()

259

Довольно много пропусков. Посмотрим, для каких моделей они возникают.

In [99]:
df[df['name'].isnull()==True].model.value_counts()

X6_M    97
X5_M    60
M5      25
M3      19
M6      15
Z4      10
I3       6
M4       5
I8       5
M2       4
Z3       4
M8       2
X4_M     2
X3_M     2
Z1       1
Z8       1
Z4_M     1
Name: model, dtype: int64

Посмотрим, сколько всего значений в датасете автомобиля модели X6_M

In [100]:
df[df['model']=='X6_M'].model.value_counts()

X6_M    97
Name: model, dtype: int64

Можем сделать вывод, что для некоторых моделей автомобилей поле name не заполнено. Можем пока что заполнить это поле названием модели.

In [101]:
df.loc[df['name'].isnull()==True, 'name'] = df.model

#### dispacement
Объем двигателя

In [102]:
df['engineDisplacement'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('displacement'))

In [103]:
df['engineDisplacement'].value_counts()

2993    1232
1995    1229
2979     740
1997     685
4395     474
2996     332
2497     318
1998     231
2494     223
1991     176
1596     144
4398     131
4799     130
1598     127
2998      81
2171      71
1499      67
2793      50
1796      48
1895      31
3498      17
4000      14
3999      14
2926      14
4999      12
3430      12
3600      11
5972      10
4619       7
2498       7
1990       6
1800       6
2997       6
6592       5
5379       4
0          4
1599       4
2491       3
1951       3
3246       3
4988       3
2986       3
1996       2
3982       2
1496       2
4941       2
647        2
1766       2
4423       2
1665       1
3795       1
3453       1
2990       1
2788       1
2495       1
2693       1
2479       1
2443       1
1573       1
Name: engineDisplacement, dtype: int64

Удалим записи с значением '0' и '647' т.к. таких значений нет в тестовой выборке. Сколько таких значений?

In [104]:
len(df[df.engineDisplacement<1000])

6

Всего 6 записей. Удалим их.

In [105]:
df = df[df.engineDisplacement>1000]

Остается довольно много значений, приведем их к виду как в тесовой выборке.

In [106]:
dic_engineDisplacement = {
    1496: 1.5,
    1499: 1.5,
    1573: 1.6,
    1596: 1.6,
    1598: 1.6,
    1599: 1.6,
    1665: 1.7,
    1766: 1.8,
    1796: 1.8,
    1800: 1.8,
    1895: 1.9,
    1951: 2.0,
    1971: 2.0,
    1990: 2.0,
    1991: 2.0,
    1995: 2.0,
    1996: 2.0,
    1997: 2.0,
    1998: 2.0,
    2000: 2.0,
    2171: 2.2,
    2443: 2.5,
    2479: 2.5,
    2491: 2.5,
    2494: 2.5,
    2495: 2.5,
    2497: 2.5,
    2498: 2.5,
    2693: 2.7,
    2788: 2.8,
    2793: 2.8,
    2926: 2.9,
    2979: 3.0,
    2986: 3.0,
    2990: 3.0,
    2993: 3.0,
    2996: 3.0,
    2997: 3.0,
    2998: 3.0,
    3210: 3.2,
    3246: 3.2,
    3430: 3.4,
    3453: 3.5,
    3498: 3.5,
    3600: 3.6,
    3795: 3.8,
    3982: 4.0,
    3999: 4.0,
    4000: 4.0,
    4395: 4.4,
    4398: 4.4,
    4423: 4.4,
    4619: 4.6,
    4799: 4.8,
    4941: 4.9,
    4988: 5.0,
    4999: 5.0,
    5379: 5.4,
    5972: 6.0,
    6592: 6.6,
}
df['engineDisplacement'] = df['engineDisplacement'].map(dic_engineDisplacement)

In [107]:
df['engineDisplacement'].value_counts()

3.0    2395
2.0    2332
4.4     607
2.5     554
1.6     276
4.8     130
2.2      71
1.5      69
1.8      56
2.8      51
1.9      31
4.0      30
3.5      18
5.0      15
2.9      14
3.4      12
3.6      11
6.0      10
4.6       7
6.6       5
5.4       4
3.2       3
4.9       2
3.8       1
2.7       1
1.7       1
Name: engineDisplacement, dtype: int64

Есть ли пропуски?

In [108]:
df['engineDisplacement'].isnull().sum()

0

#### engine_type
Тип топлива

In [109]:
df['fuelType'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('engine_type'))

Приведем значения к уже имеющимся на kaggle.

In [110]:
df['fuelType'].value_counts()

GASOLINE    4585
DIESEL      2108
HYBRID        13
Name: fuelType, dtype: int64

In [111]:
dic_fuelType = {
    'DIESEL': 'diesel',
    'GASOLINE': 'petrol',
    'HYBRID': 'hybrid',
    'ELECTRO': 'electro'
}
df['fuelType'] = df['fuelType'].map(dic_fuelType)

Пропуски?

In [112]:
df['fuelType'].isnull().sum()

0

#### gear_type
Привод

In [113]:
df['drive'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('gear_type'))

In [114]:
df.drive.value_counts()

ALL_WHEEL_DRIVE    3871
REAR_DRIVE         2814
FORWARD_CONTROL      21
Name: drive, dtype: int64

Приведем значения к уже имеющимся на kaggle.

In [115]:
dic_drive = {
    'ALL_WHEEL_DRIVE': '4wd',
    'REAR_DRIVE': 'rwd',
    'FORWARD_CONTROL': 'fwd'
}
df['drive'] = df['drive'].map(dic_drive)

Пропуски

In [116]:
df['drive'].isnull().sum()

0

#### transmission
Тип коробки передач

In [117]:
df['vehicleTransmission'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('transmission'))

In [118]:
df.vehicleTransmission.value_counts()

AUTOMATIC     6175
MECHANICAL     459
ROBOT           72
Name: vehicleTransmission, dtype: int64

Приведем значения к уже имеющимся на kaggle.

In [119]:
dic_vehicleTransmission = {
    'AUTOMATIC': 'AT',
    'MECHANICAL': 'MT',
    'ROBOT': 'AM'
}
df['vehicleTransmission'] = df['vehicleTransmission'].map(dic_vehicleTransmission)

Пропуски

In [120]:
df['vehicleTransmission'].isnull().sum()

0

#### power
Мощность двигателя

In [121]:
df['enginePower'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('power'))

In [122]:
df['enginePower'].value_counts()

184    813
306    457
249    441
218    432
245    415
      ... 
480      1
75       1
610      1
230      1
449      1
Name: enginePower, Length: 78, dtype: int64

Тут видимо будем тоже на категории делить.

Пропуски

In [123]:
df['enginePower'].isnull().sum()

0

#### acceleration
Разгон до 100 км/ч

In [124]:
df['acceleration'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('acceleration'))

In [125]:
df['acceleration'].value_counts()

8.1     404
7.6     352
6.7     316
7.5     311
6.5     255
       ... 
13.0      1
12.1      1
12.6      1
12.8      1
11.1      1
Name: acceleration, Length: 98, dtype: int64

Здесь можно будет сократить количество параметров, например, округлив до целого.

In [126]:
df['acceleration'].isnull().sum()

22

Пропусков мало, заменим их на среднее значение.

In [127]:
df.loc[df['acceleration'].isnull()==True, 'acceleration'] = df['acceleration'].mean()

#### fuelRate
Расход топлива

In [128]:
df['fuelRate'] = df['vehicle_info'].apply(lambda x: x.get('tech_param').get('fuel_rate'))

In [129]:
df['fuelRate'].value_counts()

6.2     319
10.1    249
6.0     225
5.6     202
10.3    178
       ... 
11.5      2
3.3       1
14.5      1
13.4      1
12.7      1
Name: fuelRate, Length: 99, dtype: int64

Здесь можно будет сократить количество параметров, например, округлив до целого.

Пропуски

In [130]:
df['fuelRate'].isnull().sum()

465

Много пропусков. Не думаю, что параметр сильно важный. Заменим на среднее значение.

In [131]:
df.loc[df['fuelRate'].isnull()==True, 'fuelRate'] = df['fuelRate'].mean()

Удалим 'vehicle_info' т.к. все необходимое забрали. Возможно стоило что-то ещё вытянуть, подумаем над этим, если останется время.

In [132]:
df = df.drop('vehicle_info', axis=1)

# Итоги
Посмотрим на получившийся датасет

In [133]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6706 entries, 0 to 6752
Data columns (total 25 columns):
color                  6706 non-null object
rating                 6706 non-null float64
priceDiff              6706 non-null float64
owners                 6706 non-null float64
technicalPassport      6706 non-null float64
age                    6706 non-null int64
taxPrice               6706 non-null int64
insurancePrice         6706 non-null int64
price                  6706 non-null int64
mileage                6706 non-null int64
bodyType               6706 non-null object
numberOfDoors          6706 non-null object
autoClass              6706 non-null object
equipment              6706 non-null object
tags                   6706 non-null object
model                  6706 non-null object
modelAge               6706 non-null int64
name                   6706 non-null object
engineDisplacement     6706 non-null float64
fuelType               6706 non-null object
drive         

,color,rating,priceDiff,owners,technicalPassport,age,taxPrice,insurancePrice,price,mileage,...,model,modelAge,name,engineDisplacement,fuelType,drive,vehicleTransmission,enginePower,acceleration,fuelRate
0,black,4.6,-14.0,3.0,1.0,16,14170,6309,665000,195000,...,X5,17,3.0d,3.0,diesel,4wd,AT,218,8.8,7.985547
1,black,4.8,-6.0,3.0,1.0,9,9200,6309,899000,219000,...,5ER,11,520,2.0,diesel,rwd,AT,184,8.1,5.200000
2,grey,4.6,-11.0,3.0,1.0,13,53250,6309,790000,155000,...,X5,14,48i,4.8,petrol,4wd,AT,355,6.5,12.000000
3,brown,4.6,-14.0,3.0,1.0,16,14170,6309,530000,348000,...,X5,17,3.0d,3.0,diesel,4wd,AT,218,8.8,7.985547
4,black,4.6,-6.0,2.0,1.0,10,38700,6309,1100000,150000,...,7ER,12,730L,3.0,petrol,rwd,AT,258,7.8,9.800000


Итого у нас осталось 25 признаков, включая целевой. Продолжим работу с выбросами и созданием новых признаков в ноутбуке на kaggle.

In [138]:
df.to_csv('autoru.csv')